### Setup

In [1]:
# Main
import numpy as np
import pandas as pd

# Unit root
from quantileADF import QADF
from quantileTAR import QTAR, QTAR_CustomRport
from report import countryReport, reportCountries

# Utilities
import warnings
from statsmodels.tools.sm_exceptions import IterationLimitWarning
warnings.simplefilter('ignore', IterationLimitWarning)
pd.set_option('display.max_rows', 500)

# Setting up Urls
repoUrl = 'https://raw.githubusercontent.com/vlad-yeghiazaryan/QADF/master/data/'
datasetUrl = repoUrl + 'dataset.csv'
broadREERsUrl = repoUrl + 'broadREERs.csv'
narrowREERsUrl = repoUrl + 'narrowREERs.csv'

In [2]:
broadREERs = pd.read_csv(broadREERsUrl, index_col=0, parse_dates=True)
narrowREERs = pd.read_csv(narrowREERsUrl, index_col=0, parse_dates=True)

# Extracting country information
countries = ['Armenia', 'Belarus', 'Kazakhstan', 'Kyrgyz Republic',
             'Moldova', 'Russian Federation', 'Tajikistan', 'Ukraine']

startDate = '2000-01-01'
startDateWithoutTaj = '1995-01-01'

dataNarrow =  narrowREERs[countries].loc[startDate:]
dataNarrow.index.freq = 'MS'

data = broadREERs[countries].loc[startDate:]
data.index.freq = 'MS'

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2000-01-01 to 2020-11-01
Freq: MS
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Armenia             251 non-null    float64
 1   Belarus             251 non-null    float64
 2   Kazakhstan          251 non-null    float64
 3   Kyrgyz Republic     251 non-null    float64
 4   Moldova             251 non-null    float64
 5   Russian Federation  251 non-null    float64
 6   Tajikistan          251 non-null    float64
 7   Ukraine             251 non-null    float64
dtypes: float64(8)
memory usage: 17.6 KB


In [3]:
# renormalize REERs at 2000q1 = 100
data = 100*data/data.iloc[0,:]

# Changing data
dataLogs = data.apply(np.log, axis=1)
dataPtC = dataLogs.diff()[1:]

### Report

In [4]:
dataLogs.head(2)

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Moldova,Russian Federation,Tajikistan,Ukraine
Date,,,,,,,,
2000-01-01,4.605170,4.605170,4.605170,4.605170,4.605170,4.605170,4.605170,4.605170
2000-02-01,4.618549,4.503724,4.613256,4.612896,4.602628,4.616694,4.599413,4.620071


In [5]:
levels=[0.01,0.05,0.1]
quantiles = np.arange(0.1, 1, 0.1)
drop = ['δ²', 'Lags', 'ρ₁(OLS)', 'CV1%', 'CV5%', 'CV10%']
qadfParams =  {'model':'ct', 'pmax':12, 'ic':'AIC'}
reps = 300

In [6]:
# Broad EAEU logs
logResults, logReport = reportCountries(dataLogs, QADF, qadfParams, quantiles, 
                                        reps, levels, dropColumns=drop)
logReport.to_excel('Outputs/logReport(C&T).xlsx', float_format="%.3f")

Starting Execution:

Armenia finished in: 181.82s
Belarus finished in: 284.21s
Kazakhstan finished in: 169.43s
Kyrgyz Republic finished in: 172.95s
Moldova finished in: 188.4s
Russian Federation finished in: 221.39s
Tajikistan finished in: 178.15s
Ukraine finished in: 176.24s

Total time spent executing: 26m, 13s


In [7]:
logReport

quantile                                          0.1             0.2  \
Countries          Variable/Quantile                                    
Armenia            α₀(τ)                  0.24(0.133)     0.21(0.123)   
                   ρ₁(τ)                  0.94(0.123)     0.95(0.127)   
                   Half-lives (years)               1               1   
                   tₙ(τ)                 -2.41(0.103)   -3.2(0.033)**   
                   QKS                      3.2(0.17)                   
Belarus            α₀(τ)                   0.5(0.097)    0.44(0.02)**   
                   ρ₁(τ)                 0.89(0.097)*     0.9(0.02)**   
                   Half-lives (years)               0               1   
                   tₙ(τ)                 -1.68(0.127)   -2.88(0.02)**   
                   QKS                   5.54(0.0)***                   
Kazakhstan         α₀(τ)                    0.12(0.4)     0.02(0.183)   
                   ρ₁(τ)                   0.97(0.42)     0.99(0.837)   
                   Half-lives (years)               2              11   
                   tₙ(τ)                  -1.1(0.383)    -0.48(0.833)   
                   QKS                      1.9(0.65)                   
Kyrgyz Republic    α₀(τ)                 0.41(0.033)*     0.23(0.113)   
                   ρ₁(τ)                0.91(0.033)**      0.95(0.12)   
                   Half-lives (years)               1               1   
                   tₙ(τ)                -2.39(0.093)*    -2.35(0.133)   
                   QKS                    2.39(0.503)                   
Moldova            α₀(τ)                  0.27(0.107)     0.21(0.157)   
                   ρ₁(τ)                  0.94(0.107)     0.95(0.167)   
                   Half-lives (years)               1               1   
                   tₙ(τ)                 -2.61(0.06)*     -2.38(0.14)   
                   QKS                    2.61(0.387)                   
Russian Federation α₀(τ)                -0.19(0.0)***   -0.17(0.0)***   
                   ρ₁(τ)                    1.04(1.0)       1.03(1.0)   
                   Half-lives (years)               ∞               ∞   
                   tₙ(τ)                    1.09(1.0)       2.13(1.0)   
                   QKS                  4.8(0.003)***                   
Tajikistan         α₀(τ)                 0.66(0.0)***  0.35(0.003)***   
                   ρ₁(τ)               0.85(0.003)***  0.92(0.007)***   
                   Half-lives (years)               0               1   
                   tₙ(τ)               -3.41(0.013)**   -2.62(0.05)**   
                   QKS                   3.41(0.067)*                   
Ukraine            α₀(τ)                 -0.03(0.083)     0.08(0.383)   
                   ρ₁(τ)                   1.0(0.937)     0.98(0.427)   
                   Half-lives (years)               ∞               3   
                   tₙ(τ)                   0.06(0.94)     -0.86(0.55)   
                   QKS                  3.88(0.017)**                   

quantile                                          0.3             0.4  \
Countries          Variable/Quantile                                    
Armenia            α₀(τ)                  0.09(0.457)     0.04(0.227)   
                   ρ₁(τ)                   0.98(0.43)     0.99(0.763)   
                   Half-lives (years)               3               6   
                   tₙ(τ)                  -1.32(0.54)    -0.59(0.843)   
                   QKS                                                  
Belarus            α₀(τ)                 0.47(0.01)**   0.41(0.007)**   
                   ρ₁(τ)                 0.9(0.017)**   0.91(0.01)***   
                   Half-lives (years)               1               1   
                   tₙ(τ)                -4.45(0.0)***   -5.26(0.0)***   
                   QKS                                                  
Kazakhstan         α₀(τ)                   0.04(0.34)    

In [12]:
EAEU = pd.Series((dataLogs.index > '2014-12-01').astype(int), index=dataLogs.index, name='EAEU')
levels=[0.01,0.05,0.1]
quantiles = np.arange(0.1, 1, 0.1)
drop = ['δ²', 'Lags', 'CV1%', 'CV5%', 'CV10%']
qtarParams =  {'threshold_dummy':EAEU, 'model':'ct', 'pmax':12, 'ic':'AIC'}
reps = 300

In [13]:
QTAR_logResults, QTAR_logReport = reportCountries(dataLogs, QTAR, qtarParams, quantiles, reps, levels, 
                                                  customReport=QTAR_CustomRport, dropColumns=drop)
QTAR_logReport.to_excel('Outputs/QTAR_logReport(C&T).xlsx', float_format="%.3f")

Starting Execution:

Armenia finished in: 205.49s
Belarus finished in: 319.19s
Kazakhstan finished in: 199.46s
Kyrgyz Republic finished in: 196.26s
Moldova finished in: 200.09s
Russian Federation finished in: 252.24s
Tajikistan finished in: 200.04s
Ukraine finished in: 196.15s

Total time spent executing: 29m, 29s


In [14]:
QTAR_logReport

quantile                                            0.1               0.2  \
Countries          Variable/Quantile                                        
Armenia            α₀(τ)                    0.26(0.213)       0.22(0.187)   
                   pre-ρ₁(τ)                 0.94(0.21)       0.95(0.193)   
                   post-ρ₂(τ)                0.94(0.22)         0.95(0.2)   
                   pre-ρ₁(OLS)              0.97(0.567)       0.97(0.567)   
                   post-ρ₂(OLS)              0.96(0.56)        0.96(0.56)   
                   ρ₁(τ)-ρ₂(τ)             0.0001(0.51)     0.0006(0.637)   
                   Half-lives (years)                 1                 1   
                   pre-tₙ(τ)               -37.02(0.21)      -50.78(0.12)   
                   post-tₙ(τ)             -35.93(0.217)     -49.81(0.137)   
                   pre-QKS                  50.78(0.35)                     
                   post-QKS                49.81(0.363)                     
Belarus            α₀(τ)                    0.45(0.183)     0.51(0.023)**   
                   pre-ρ₁(τ)                 0.9(0.207)     0.89(0.023)**   
                   post-ρ₂(τ)                0.9(0.197)     0.88(0.023)**   
                   pre-ρ₁(OLS)                0.9(0.16)         0.9(0.16)   
                   post-ρ₂(OLS)              0.9(0.133)        0.9(0.133)   
                   ρ₁(τ)-ρ₂(τ)             0.0014(0.67)     0.0032(0.927)   
                   Half-lives (years)                 1                 0   
                   pre-tₙ(τ)              -22.45(0.297)   -68.83(0.037)**   
                   post-tₙ(τ)             -22.72(0.303)   -70.61(0.033)**   
                   pre-QKS                99.6(0.017)**                     
                   post-QKS             101.14(0.017)**                     
Kazakhstan         α₀(τ)                     0.33(0.07)      0.31(0.01)**   
                   pre-ρ₁(τ)                0.92(0.07)*     0.93(0.013)**   
                   post-ρ₂(τ)              0.92(0.053)*    0.92(0.007)***   
                   pre-ρ₁(OLS)             0.94(0.097)*      0.94(0.097)*   
                   post-ρ₂(OLS)             0.93(0.07)*       0.93(0.07)*   
                   ρ₁(τ)-ρ₂(τ)            0.0086(0.987)      0.007(0.997)   
                   Half-lives (years)                 1                 1   
                   pre-tₙ(τ)               -17.61(0.32)  -73.46(0.003)***   
                   post-tₙ(τ)              -18.4(0.287)  -75.87(0.003)***   
                   pre-QKS               81.34(0.023)**                     
                   post-QKS              83.83(0.023)**                     
Kyrgyz Republic    α₀(τ)                    0.47(0.04)*      0.34(0.043)*   
                   pre-ρ₁(τ)               0.89(0.04)**     0.92(0.043)**   
                   post-ρ₂(τ)             0.89(0.037)**      0.92(0.04)**   
                   pre-ρ₁(OLS)              0.95(0.303)       0.95(0.303)   
                   post-ρ₂(OLS)             0.95(0.267)       0.95(0.267)   
                   ρ₁(τ)-ρ₂(τ)             0.0029(0.91)     0.0018(0.867)   
                   Half-lives (years)                 1                 1   
                   pre-tₙ(τ)             -66.37(0.073)*   -78.98(0.043)**   
                   post-tₙ(τ)            -66.68(0.063)*   -79.07(0.043)**   
                   pre-QKS                 83.17(0.143)                     
                   post-QKS                84.51(0.143)                     
Moldova            α₀(τ)                    0.29(0.153)       0.32(0.063)   
                   pre-ρ₁(τ)                0.93(0.153)      0.93(0.063)*   
                   post-ρ₂(τ)               0.93(0.147)      0.93(0.053)*   
                   pre-ρ₁(OLS)              0.96(0.347)       0.96(0.347)   
                   post-ρ₂(OLS)             0.96(0.307)       0.96(0.307)   
                   ρ₁(τ)-ρ₂(τ)            0.0025(0.887)      0.0028(0.95)

In [18]:
qtar_model = QTAR(dataLogs['Ukraine'], **qtarParams)
res = qtar_model.fit(0.6)
res.regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         QuantReg Regression Results                          
==============================================================================
Dep. Variable:                Ukraine   Pseudo R-squared:               0.8105
Model:                       QuantReg   Bandwidth:                     0.01266
Method:                 Least Squares   Sparsity:                      0.04497
Date:                Fri, 02 Jul 2021   No. Observations:                  248
Time:                        01:43:07   Df Residuals:                      242
                                        Df Model:                            5
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.3270      0.071      4.625      0.000       0.188       0.466
pre-EAEU_Ukraine.L1      0.9327      0.015     63.332      0.000       0.904       0.962
post-EAEU_Ukraine.L1     0.9304      0.016     59.901      0.000       0.900       0.961
Diff.L1                  0.1411      0.039      3.615      0.000       0.064       0.218
Diff.L2                 -0.1489      0.036     -4.157      0.000      -0.220      -0.078
trend                  8.94e-06   3.21e-05      0.278      0.781   -5.43e-05    7.22e-05
========================================================================================

The condition number is large, 7.33e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
# Please ignore the cirtical values
res.summary()

quantile: 0.6
Lags: 2
α₀(τ): 0.327
pre-ρ₁(τ): 0.9327
post-ρ₂(τ): 0.9304
pre-ρ₁(OLS): 0.943
post-ρ₂(OLS): 0.9432
ρ₁(τ)-ρ₂(τ): 0.00234
δ²: 0.227
Half-lives: 9.607
pre-tₙ(τ): -56.9026
post-tₙ(τ): -55.8753
CV10%: -2.2477
CV5%: -2.5965
CV1%: -3.2411
